In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas
od.download("https://www.kaggle.com/datasets/apollo2506/eurosat-dataset",force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: prathyushamatta
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/apollo2506/eurosat-dataset


100%|██████████| 2.04G/2.04G [00:27<00:00, 80.7MB/s]


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# Define the root directory for EuroSAT dataset
root_dir = "/content/EuroSAT"

# Define transformations (including normalization, resizing, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit pretrained models like ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Set your dataset root directory
root_dir = "/content/EuroSAT"
classes = os.listdir(root_dir)

# Directories where train and validation data will be stored
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

# Create train and validation directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for class_name in classes:
    class_path = os.path.join(root_dir, class_name)
    if os.path.isdir(class_path):
        # Create class directories in train and val folders
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

        # Get all images for the class
        images = os.listdir(class_path)
        train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

        # Move train images
        for img in train_images:
            shutil.move(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))

        # Move validation images
        for img in val_images:
            shutil.move(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("Dataset split into train and validation folders.")


Dataset split into train and validation folders.


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Load a pretrained ResNet model
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for multiclass classification (EuroSAT has 10 classes)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Track loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print training statistics
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%')

        # Validation step (you can also add early stopping, etc.)
        validate(model, val_loader, criterion)

# Validation loop
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {100 * correct/total:.2f}%')

# Train the model
train(model, train_loader, val_loader, criterion, optimizer, epochs=10)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]


Epoch 1/10, Loss: 0.5051, Accuracy: 84.17%
Validation Loss: 0.7210, Validation Accuracy: 82.00%
Epoch 2/10, Loss: 0.2458, Accuracy: 91.96%
Validation Loss: 0.2569, Validation Accuracy: 91.87%
Epoch 3/10, Loss: 0.1930, Accuracy: 93.54%
Validation Loss: 0.2170, Validation Accuracy: 92.63%
Epoch 4/10, Loss: 0.1548, Accuracy: 94.88%
Validation Loss: 0.1870, Validation Accuracy: 93.24%
Epoch 5/10, Loss: 0.1320, Accuracy: 95.49%
Validation Loss: 0.1403, Validation Accuracy: 95.11%
Epoch 6/10, Loss: 0.1203, Accuracy: 95.94%
Validation Loss: 0.2165, Validation Accuracy: 93.50%
Epoch 7/10, Loss: 0.1108, Accuracy: 96.01%
Validation Loss: 0.0809, Validation Accuracy: 97.54%
Epoch 8/10, Loss: 0.1102, Accuracy: 96.28%
Validation Loss: 0.1191, Validation Accuracy: 96.26%
Epoch 9/10, Loss: 0.0925, Accuracy: 96.81%
Validation Loss: 0.1060, Validation Accuracy: 96.37%
Epoch 10/10, Loss: 0.0780, Accuracy: 97.31%
Validation Loss: 0.1033, Validation Accuracy: 96.52%


In [ ]:
torch.save(model.state_dict(), 'fine_tuned_model.pth')

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define the image transformations (the same as used during training)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standard normalization
])

# Load the saved model
model = models.resnet50(pretrained=False)
num_classes = 10  # EuroSAT has 10 classes
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load('fine_tuned_model.pth'))  # Load fine-tuned model weights
model = model.to(device)  # Move the model to GPU if available
model.eval()  # Set model to evaluation mode

# Function to predict the image label
def predict_image_label(image_path):
    # Load the image and preprocess it
    img = Image.open(image_path)
    img_tensor = preprocess(img).unsqueeze(0).to(device)  # Add batch dimension and move to GPU if available

    # Make predictions
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    # Mapping index to class label (assuming you have a list of class names)
    class_names = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
                   'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']

    predicted_label = class_names[predicted.item()]
    return predicted_label

# Prompt user for image input
def main():
    # Ask the user to input the image path
    image_path = input("Please enter the path to the image: ")

    try:
        predicted_label = predict_image_label(image_path)
        print(f'Predicted label for the input image: {predicted_label}')
    except Exception as e:
        print(f"Error processing the image: {e}")

# Run the prompt to get user input
if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-13-0d491ba52209>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomme

Please enter the path to the image: /content/EuroSAT/val/Highway/Highway_100.jpg
Predicted label for the input image: Highway
